In [1]:
from IPython.display import display, HTML
import warnings

In [2]:
import mlflow

# Print the MLflow version
print(mlflow.__version__)

2.3.2


In [3]:
from mlflow.tracking import MlflowClient

In [4]:
from mlflow.entities import ViewType

In [22]:
from datetime import datetime

In [5]:
cd ..

C:\Users\KonuTech\PycharmProjects\mlops-zoomcamp-training


In [6]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [7]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [8]:
# Get the list of available methods and attributes
available_methods = dir(client)

# Print the list
for method in available_methods:
    print(method)

__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
_get_registry_client
_log_artifact_helper
_record_logged_model
_registry_uri
_tracking_client
create_experiment
create_model_version
create_registered_model
create_run
delete_experiment
delete_model_version
delete_model_version_tag
delete_registered_model
delete_registered_model_alias
delete_registered_model_tag
delete_run
delete_tag
download_artifacts
get_experiment
get_experiment_by_name
get_latest_versions
get_metric_history
get_model_version
get_model_version_by_alias
get_model_version_download_uri
get_model_version_stages
get_registered_model
get_run
list_artifacts
log_artifact
log_artifacts
log_batch
log_dict
log_figure
log_image
log_metric
log_param
log_text
rename_experiment
rename_registered_model
restore_experiment

### Interacting with the MLflow tracking server

In [9]:
client.search_experiments(order_by=["experiment_id DESC"])

[<Experiment: artifact_location='file:///C:/Users/KonuTech/PycharmProjects/mlops-zoomcamp-training/mlruns/2', creation_time=1684388799134, experiment_id='2', last_update_time=1684388799134, lifecycle_stage='active', name='duration-prediction', tags={}>,
 <Experiment: artifact_location='file:///C:/Users/KonuTech/PycharmProjects/mlops-zoomcamp-training/mlruns/1', creation_time=1684307915748, experiment_id='1', last_update_time=1684307915748, lifecycle_stage='active', name='duration-prediction-homework', tags={}>]

In [11]:
client.create_experiment(name="my-cool-experiment")

'3'

In [12]:
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [14]:
runs

[<Run: data=<RunData: metrics={'rmse': 4.791062660576858}, params={'learning_rate': '0.2027270019188835',
  'max_depth': '11',
  'min_child_weight': '9.498814078399025',
  'objective': 'reg:linear',
  'reg_alpha': '0.05903805330249986',
  'reg_lambda': '0.026935158747484356',
  'seed': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "80fce15eb4244fd4b6ac3a4d0f2290c3", '
                              '"artifact_path": "models_mlflow", '
                              '"utc_time_created": "2023-05-17 '
                              '19:51:16.464479", "flavors": {"python_function": '
                              '{"loader_module": "mlflow.xgboost", '
                              '"python_version": "3.9.3", "data": "model.xgb", '
                              '"env": {"conda": "conda.yaml", "virtualenv": '
                              '"python_env.yaml"}}, "xgboost": {"xgb_version": '
                              '"1.7.5", "data": "model.xgb", "model_class": '
                    

In [13]:
for r, run in enumerate(runs):
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 80fce15eb4244fd4b6ac3a4d0f2290c3, rmse: 4.7911
run id: 0b14297532c343b19eeb5ec1f422f5ad, rmse: 4.7911
run id: 5212d0d698c046d1b8bc7461dc4f658c, rmse: 4.7911
run id: 361f45fff7e54aef8b45e42e5e285db3, rmse: 4.7931
run id: 726a6c5750aa471d893948dcd804f82c, rmse: 4.7931


### Interacting with the Model Registry

In [15]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [16]:
run_id = "7aa27c3f85b9474d811bba5b9c7d4002"
model_uri = f"runs:/{run_id}/models_mlflow"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/05/18 14:23:48 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 4
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1684412628922, current_stage='None', description=None, last_updated_timestamp=1684412628922, name='nyc-taxi-regressor', run_id='7aa27c3f85b9474d811bba5b9c7d4002', run_link=None, source='file:///C:/Users/KonuTech/PycharmProjects/mlops-zoomcamp-training/mlruns/2/7aa27c3f85b9474d811bba5b9c7d4002/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [17]:
client.list_artifacts(run_id)

[<FileInfo: file_size=None, is_dir=True, path='models_mlflow'>,
 <FileInfo: file_size=None, is_dir=True, path='preprocessor'>]

In [18]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

In [19]:
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: Staging
version: 2, stage: Archived
version: 4, stage: None


In [20]:
model_version = 4
new_stage = "Staging"

In [21]:
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1684412628922, current_stage='Staging', description=None, last_updated_timestamp=1684412915609, name='nyc-taxi-regressor', run_id='7aa27c3f85b9474d811bba5b9c7d4002', run_link=None, source='file:///C:/Users/KonuTech/PycharmProjects/mlops-zoomcamp-training/mlruns/2/7aa27c3f85b9474d811bba5b9c7d4002/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [23]:
date = datetime.today().date()

In [24]:
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1684412628922, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2023-05-18', last_updated_timestamp=1684413002654, name='nyc-taxi-regressor', run_id='7aa27c3f85b9474d811bba5b9c7d4002', run_link=None, source='file:///C:/Users/KonuTech/PycharmProjects/mlops-zoomcamp-training/mlruns/2/7aa27c3f85b9474d811bba5b9c7d4002/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=4>

### Comparing versions and selecting the new "Production" model